In [18]:
import os
import sys
import pandas as pd
from pathlib import Path
from datetime import datetime
from model_settings import ms
notebook_dir = str(Path().resolve())

from historical_av_underlying_fetcher import spots, symbol

In [19]:
dates = spots.index
dates = dates[:3]
for date in dates:
    print(date)

2021-03-03
2021-03-02
2021-03-01


In [20]:
from historical_alphaVantage_collection import collect_av_link
chain = {}
for date in dates:
    spot = float(spots[date])
    link = collect_av_link(date)
    printdate = datetime.strptime(date, '%Y-%m-%d').strftime('%A, %Y-%m-%d')
    chain[date] = link
for key in chain.keys():
    print(key)

2021-03-03
2021-03-02
2021-03-01


In [24]:
contracts = []
for key,link in chain.items():
    contracts.append(link['raw_data'])
contracts = pd.concat(contracts,ignore_index=True)
contracts

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,SPY210303C00190000,SPY,2021-03-03,190.00,call,0.00,191.56,191.18,11,191.93,11,0,0,2021-03-03,5.28556,0.99606,0.00011,-2.25171,0.00233,0.00516
1,SPY210303P00190000,SPY,2021-03-03,190.00,put,0.00,0.01,0.00,0,0.01,7724,0,0,2021-03-03,4.13574,-0.00044,0.00002,-0.23699,0.00031,-0.00000
2,SPY210303C00195000,SPY,2021-03-03,195.00,call,0.00,186.56,186.18,11,186.93,11,0,0,2021-03-03,5.10263,0.99592,0.00012,-2.24258,0.00241,0.00530
3,SPY210303P00195000,SPY,2021-03-03,195.00,put,0.00,0.01,0.00,0,0.01,7716,0,0,2021-03-03,3.98925,-0.00045,0.00002,-0.23633,0.00032,-0.00001
4,SPY210303C00200000,SPY,2021-03-03,200.00,call,0.00,181.56,181.18,11,181.93,11,0,0,2021-03-03,4.92413,0.99578,0.00013,-2.23288,0.00248,0.00543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29985,SPY231215P00580000,SPY,2023-12-15,580.00,put,0.00,200.64,198.28,200,203.00,1,0,0,2021-03-01,0.21388,-0.82367,0.00186,-0.06093,1.68603,-14.55948
29986,SPY231215C00585000,SPY,2023-12-15,585.00,call,4.35,4.48,3.00,75,5.97,2000,2,54,2021-03-01,0.16830,0.09712,0.00157,-0.03394,1.11823,0.93122
29987,SPY231215P00585000,SPY,2023-12-15,585.00,put,0.00,205.23,202.96,200,207.50,1,0,0,2021-03-01,0.21449,-0.82887,0.00182,-0.05985,1.65419,-14.74471
29988,SPY231215C00590000,SPY,2023-12-15,590.00,call,4.00,4.29,4.00,1,4.59,12,2,216,2021-03-01,0.16907,0.09323,0.00151,-0.03308,1.08504,0.89430


In [ ]:
chain